In [1]:
import pandas as pd; import numpy as np; import janitor as jn; from pandas import Timedelta

In [2]:
from string import ascii_lowercase
np.random.seed(1)
n = 20; k = 30

idx1 = np.random.randint(0, high = 100, size = n)
idx2 = np.random.randint(0, high = 100, size = n)

d1 = dict(x = np.random.choice(list(ascii_lowercase[:5]), size=n),
          start = np.minimum(idx1, idx2),
          end = np.maximum(idx1, idx2))

d2 = dict(x = np.random.choice(list(ascii_lowercase[:15]), size=k),
          pos1 = np.random.randint(low=60, high = 151, size=k))

d1 = pd.DataFrame(d1)
d2 = pd.DataFrame(d2)

In [3]:
bla = (d1
.conditional_join(
    d2, 
    ('start', 'pos1', '>='), 
    ('end', 'pos1', '>='), 
   # ('xx', 'xx', '=='), 
    use_numba=True)
)

bla

[(array([4, 6, 7]),
  array([21, 13, 10, 17, 11]),
  array([25, 25, 27]),
  array([0, 0, 1]),
  array([0, 0, 1, 1, 1])),
 (array([ 2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 17, 18, 19]),
  array([29,  4, 28,  6, 19, 27, 21, 13, 10, 17, 11]),
  array([25, 25, 22, 22, 20, 19, 22, 25, 25, 27, 29, 22, 29, 23]),
  array([4, 4, 2, 2, 1, 0, 2, 4, 4, 5, 6, 2, 6, 3]),
  array([0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 6]))]

In [4]:
pd.Series(bla[-1]).is_monotonic_decreasing

ValueError: operands could not be broadcast together with shapes (11,) (14,) 

In [ ]:
# pprint([region_left, region_right, left_region, right_region])

In [ ]:
df1 = pd.DataFrame({'id': [1,1,1,2,2,3], 
                    'value_1': [2,5,7,1,3,4]})
                    
df2 = pd.DataFrame({'id': [1,1,1,1,2,2,2,3], 
                    'value_2A': [0,3,7,12,0,2,3,1], 
                    'value_2B': [1,5,9,15,1,4,6,3]})

In [ ]:
# df1 = pd.concat([df1]*10)
# df2 = pd.concat([df2]*10)

In [ ]:
(df1
.conditional_join(
    df2,
 #   ('id', 'id', '=='),
    ('value_1', 'value_2A', '<='),
    ('value_1', 'value_2B', '<='),
    use_numba=True
))

(array([0, 1, 2, 3, 4, 5]), array([0, 4, 7, 5, 1, 6, 2, 3]))

In [ ]:
df1 = pd.concat([df1]*50)
df2 = pd.concat([df2]*50)

In [ ]:
bla = (df1
.conditional_join(
    df2,
 #   ('id', 'id', '=='),
    ('value_1', 'value_2A', '>='),
    ('value_1', 'value_2B', '<='),
    use_numba=True
))

bla

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [ ]:
pd.Series(bla[-2]).is_monotonic_decreasing

False

In [ ]:
url = 'https://raw.githubusercontent.com/samukweku/data-wrangling-blog/master/notebooks/Data_files/results.csv'
events = pd.read_csv(url, parse_dates=['start', 'end']).iloc[:, 1:]
events.head()

,id,name,audience,start,sponsor,end
0,1,Event 1,1178,2022-11-19 10:00:00,Sponsor 2,2022-11-19 10:15:00
1,2,Event 2,1446,2015-09-27 15:00:00,Sponsor 11,2015-09-27 15:11:00
2,3,Event 3,2261,2019-11-12 18:00:00,Sponsor 10,2019-11-12 18:53:00
3,4,Event 4,1471,2019-12-24 22:00:00,Sponsor 6,2019-12-24 22:11:00
4,5,Event 5,2605,2028-06-20 12:00:00,Sponsor 8,2028-06-20 12:31:00


In [ ]:
bla =(events
.conditional_join(
    events,
    ('start', 'end', '<='),
    ('end', 'start', '>='),
    ('id', 'id', '!='),
    use_numba = True,
    df_columns = ['id', 'start', 'end'],
    right_columns = ['id', 'start', 'end'])
)

bla

(array([    0,     1,     2, ..., 29996, 29997, 29998]),
 array([25296, 10726, 24103, ...,  3544, 15884, 13367]))

In [ ]:
bla

(array([    0,     1,     2, ..., 29996, 29997, 29998]),
 array([25296, 10726, 24103, ...,  3544, 15884, 13367]))

In [ ]:
pd.Series(bla[-1]).is_monotonic_decreasing

False

In [5]:
C=dict(key=[f"c{num}" for num in range(1,8)],
       vol=[35,15,5,35,18,90,17],
       profit=[45,35,55,12,15,55,11],
       unitsSold=[15,10,30,10,15,80,2]
       )
C=pd.DataFrame(C)


D=dict(key=[f'd{num}' for num in range(1,9)],
       vol=[20,50,15,16,40,20,40,2],
       profit=[30,10,12,52,35,20,30,57],
       unitsSold=[20,35,10,12,40,30,5,15]
       )
D=pd.DataFrame(D)

In [6]:
f=(C
.conditional_join(
    D, 
    ('vol','vol','<'),
    ('profit','profit','>'),
    use_numba=True)
)
f

[(array([0, 1, 2, 3, 4, 6]),
  array([2, 3, 0, 5, 4, 6, 1]),
  array([5, 2, 1, 5, 3, 3]),
  array([3, 1, 0, 3, 2, 2]),
  array([0, 1, 2, 2, 3, 3, 3])),
 (array([0, 1, 2, 3, 4, 5, 6]),
  array([3, 4, 6, 0, 5, 2, 1]),
  array([2, 3, 1, 7, 6, 1, 7]),
  array([1, 2, 0, 4, 3, 0, 4]),
  array([0, 1, 2, 2, 2, 3, 4]))]

In [ ]:
C.sort_values('vol')

,key,vol,profit,unitsSold
0,c1,35,45,15
1,c2,15,35,10
2,c3,5,55,30
3,c4,35,12,10
4,c5,18,15,15
5,c6,90,55,80
6,c7,17,11,2


In [7]:
D.sort_values('vol')
#D.sort_values('profit')
D.sort_values('profit',ascending=False)

,key,vol,profit,unitsSold
7,d8,2,57,15
3,d4,16,52,12
4,d5,40,35,40
0,d1,20,30,20
6,d7,40,30,5
5,d6,20,20,30
2,d3,15,12,10
1,d2,50,10,35


In [ ]:
D

,key,vol,profit,unitsSold
0,d1,20,30,20
1,d2,50,10,35
2,d3,15,12,10
3,d4,16,52,12
4,d5,40,35,40
5,d6,20,20,30
6,d7,40,30,5
7,d8,2,57,15


In [ ]:
C.sort_values('vol')

,key,vol,profit,unitsSold
2,c3,5,55,30
1,c2,15,35,10
6,c7,17,11,2
4,c5,18,15,15
0,c1,35,45,15
3,c4,35,12,10
5,c6,90,55,80


In [ ]:
D.sort_values('vol')

,key,vol,profit,unitsSold
7,d8,2,57,15
2,d3,15,12,10
3,d4,16,52,12
0,d1,20,30,20
5,d6,20,20,30
4,d5,40,35,40
6,d7,40,30,5
1,d2,50,10,35


In [ ]:
pd.concat([C.loc[:, ['key','vol']], D.loc[:, ['key','vol']]]).sort_values('vol',kind='stable')

,key,vol
7,d8,2
2,c3,5
1,c2,15
2,d3,15
3,d4,16
6,c7,17
4,c5,18
0,d1,20
5,d6,20
0,c1,35


In [ ]:
C.sort_values('profit',ascending=False)

,key,vol,profit,unitsSold
2,c3,5,55,30
5,c6,90,55,80
0,c1,35,45,15
1,c2,15,35,10
4,c5,18,15,15
3,c4,35,12,10
6,c7,17,11,2


In [ ]:
D.sort_values('profit',ascending=False)

,key,vol,profit,unitsSold
7,d8,2,57,15
3,d4,16,52,12
4,d5,40,35,40
0,d1,20,30,20
6,d7,40,30,5
5,d6,20,20,30
2,d3,15,12,10
1,d2,50,10,35


In [ ]:
C.vol.sort_values()

2     5
1    15
6    17
4    18
0    35
3    35
5    90
Name: vol, dtype: int64

In [ ]:
C.vol.sort_values().searchsorted(D.vol, side='left')

array([4, 6, 1, 2, 6, 4, 6, 0])

In [ ]:
D.vol

0    20
1    50
2    15
3    16
4    40
5    20
6    40
7     2
Name: vol, dtype: int64